COVID-19 in Sao Paulo State, Brazil: Automating a Thematic Map & Chart

In [ ]:
#Import Module
import arcpy

#Set up the workspace - ******change to your workspace here *********
arcpy.env.workspace = r'C:\...\sp_covid_data.gdb'

In [ ]:
#Join the CSV data with the Sao Paulo State Shapefile
sp_covid_joined_table = arcpy.AddJoin_management("sao_paulo","NM_MUN", "Dados_covid_19_municipios","Município")

arcpy.CopyFeatures_management(sp_covid_joined_table, "sp_covid")



In [ ]:
#Adding fields to the table in order to calculate the death rate for each city
#Here I am converting the fields containing the number of cases and the number of deaths to "DOUBLE" to perform the math
sp_covid = "sp_covid"
arcpy.AddField_management(sp_covid, "Deathrate", "DOUBLE")
arcpy.AddField_management(sp_covid, "num_cases", "DOUBLE")
arcpy.AddField_management(sp_covid, "num_deaths", "DOUBLE")
arcpy.management.CalculateField(sp_covid, "num_cases", '!Dados_covid_19_municipios_Mun_Total_de_casos!', "PYTHON3", '', "DOUBLE")
arcpy.management.CalculateField(sp_covid, "num_deaths", '!Dados_covid_19_municipios_Mun_Total_de_óbitos!', "PYTHON3", '', "DOUBLE")
arcpy.management.CalculateField(sp_covid, "Deathrate", '!num_deaths! / !num_cases! * 100', "PYTHON3", '', "DOUBLE")

In [ ]:
#Generating the chart

#listing the project, maps and layers to use
aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
covid_layer = map.listLayers('sp_covid')[0]


#Building the chart
c = arcpy.Chart('MyChart')
c.type = 'bar'
c.title = 'Death / Num. of Cases in Sao Paulo State - November'
c.xAxis.field = 'sao_paulo_NM_MUN'
c.yAxis.field = 'Deathrate'
c.bar.aggregation = 'SUM'
c.xAxis.title = 'City'
c.yAxis.title = 'Death %'
c.addToLayer(covid_layer)

In [ ]:
#Generating the Thematic Map

#Listing the project, map and layer to use
p = arcpy.mp.ArcGISProject("CURRENT")
m = p.listMaps("Map")[0]
lyr = m.listLayers("sp_covid")[0]
sym = lyr.symbology

#Rendering the chosen field with a color ramp
sym.updateRenderer("GraduatedColorsRenderer")
sym.renderer.classificationField = 'Deathrate'
sym.renderer.colorRamp=p.listColorRamps("Yellow-Orange-Red (Continuous)")[0]

#Breaking the class values
classBreakValues = [0,2,6,8,10]
classBreakLabels = ["div 1", "div 2", "div 3", "div 4", "div 5"]

sym.renderer.breakCount = len(classBreakValues)
count = 0

#Looping through the classes to add them to the map with the corresponding color
for brk in sym.renderer.classBreaks:
    brk.upperBound = classBreakValues[count]
    brk.label = classBreakLabels[count]
    count += 1
lyr.symbology = sym

In [ ]:
#Export the layout to a PDF file
aprx = arcpy.mp.ArcGISProject("CURRENT")
Layout = aprx.listLayouts("Layout3")[0]
#******change to your workspace here *********
Layout.exportToPDF(r'C:\.......\map_sp.pdf')